## Preprocessing

In [3]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME"])


In [5]:
#check that columsn have been effcetively dropped
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [6]:
# Determine the number of unique values in each column.
unique_values = application_df.nunique()
unique_values

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [7]:
# Look at APPLICATION_TYPE value counts for binning
application_type_value = application_df.groupby ('APPLICATION_TYPE')['APPLICATION_TYPE'].count()
application_type_value

APPLICATION_TYPE
T10      528
T12       27
T13       66
T14        3
T15        2
T17        1
T19     1065
T2        16
T25        3
T29        2
T3     27037
T4      1542
T5      1173
T6      1216
T7       725
T8       737
T9       156
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = []

# Iterate through the series
for app_name, count in application_type_value.items():
    if count < 500:
        application_types_to_replace.append(app_name)

# application_names now contains the names with counts less than 500
print(application_types_to_replace)

['T12', 'T13', 'T14', 'T15', 'T17', 'T2', 'T25', 'T29', 'T9']


In [9]:
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [10]:
# Look at CLASSIFICATION value counts for binning
class_value_types = application_df.groupby('CLASSIFICATION')['CLASSIFICATION'].count()
class_value_types

CLASSIFICATION
C0           3
C1000    17326
C1200     4837
C1230       36
C1234        2
         ...  
C7200       32
C7210        7
C8000       20
C8200       11
C8210        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [11]:
# You may find it helpful to look at CLASSIFICATION value counts >1
filtered_classifications = class_value_types[class_value_types > 1]
filtered_classifications

CLASSIFICATION
C0           3
C1000    17326
C1200     4837
C1230       36
C1234        2
C1235        9
C1237        9
C1238       10
C1240       30
C1246        2
C1250       14
C1256        2
C1257        5
C1260        3
C1267        2
C1270      114
C1278       10
C1280       50
C1300       58
C1400       34
C1500       16
C1600        5
C1700      287
C1720        6
C1800       15
C2000     6074
C2100     1883
C2300       32
C2400        6
C2700      104
C2710        3
C2800       95
C3000     1918
C3200        2
C4000      194
C4100        6
C5000      116
C6000       15
C7000      777
C7100       75
C7120       18
C7200       32
C7210        7
C8000       20
C8200       11
Name: CLASSIFICATION, dtype: int64

In [12]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

classifications_to_replace = [] #empty list to hold results

for class_name, count in class_value_types.items():   #for loop through series to ID class types with count less than 700.
  if count < 777:
    classifications_to_replace.append(class_name)

#print result for verification
print(classifications_to_replace)

['C0', 'C1230', 'C1234', 'C1235', 'C1236', 'C1237', 'C1238', 'C1240', 'C1245', 'C1246', 'C1248', 'C1250', 'C1256', 'C1257', 'C1260', 'C1267', 'C1270', 'C1278', 'C1280', 'C1283', 'C1300', 'C1370', 'C1400', 'C1500', 'C1570', 'C1580', 'C1600', 'C1700', 'C1720', 'C1728', 'C1732', 'C1800', 'C1820', 'C1900', 'C2150', 'C2170', 'C2190', 'C2300', 'C2380', 'C2400', 'C2500', 'C2561', 'C2570', 'C2600', 'C2700', 'C2710', 'C2800', 'C3200', 'C3700', 'C4000', 'C4100', 'C4120', 'C4200', 'C4500', 'C5000', 'C5200', 'C6000', 'C6100', 'C7100', 'C7120', 'C7200', 'C7210', 'C8000', 'C8200', 'C8210']


In [13]:

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [14]:
#access info for easy access to features that need to be converted to numerical
application_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   APPLICATION_TYPE        34299 non-null  object
 1   AFFILIATION             34299 non-null  object
 2   CLASSIFICATION          34299 non-null  object
 3   USE_CASE                34299 non-null  object
 4   ORGANIZATION            34299 non-null  object
 5   STATUS                  34299 non-null  int64 
 6   INCOME_AMT              34299 non-null  object
 7   SPECIAL_CONSIDERATIONS  34299 non-null  object
 8   ASK_AMT                 34299 non-null  int64 
 9   IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 2.6+ MB


In [15]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies_df = pd.get_dummies(application_df[['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']])
dummies_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,1,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [16]:
#diplay info of new df dummies_df
dummies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 42 columns):
 #   Column                        Non-Null Count  Dtype
---  ------                        --------------  -----
 0   APPLICATION_TYPE_Other        34299 non-null  uint8
 1   APPLICATION_TYPE_T10          34299 non-null  uint8
 2   APPLICATION_TYPE_T19          34299 non-null  uint8
 3   APPLICATION_TYPE_T3           34299 non-null  uint8
 4   APPLICATION_TYPE_T4           34299 non-null  uint8
 5   APPLICATION_TYPE_T5           34299 non-null  uint8
 6   APPLICATION_TYPE_T6           34299 non-null  uint8
 7   APPLICATION_TYPE_T7           34299 non-null  uint8
 8   APPLICATION_TYPE_T8           34299 non-null  uint8
 9   AFFILIATION_CompanySponsored  34299 non-null  uint8
 10  AFFILIATION_Family/Parent     34299 non-null  uint8
 11  AFFILIATION_Independent       34299 non-null  uint8
 12  AFFILIATION_National          34299 non-null  uint8
 13  AFFILIATION_Other             3

In [17]:
# concatenate the application_df and the dummies_df
application_df_processed = pd.concat([application_df[['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL']], dummies_df], axis=1)
application_df_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 45 columns):
 #   Column                        Non-Null Count  Dtype
---  ------                        --------------  -----
 0   STATUS                        34299 non-null  int64
 1   ASK_AMT                       34299 non-null  int64
 2   IS_SUCCESSFUL                 34299 non-null  int64
 3   APPLICATION_TYPE_Other        34299 non-null  uint8
 4   APPLICATION_TYPE_T10          34299 non-null  uint8
 5   APPLICATION_TYPE_T19          34299 non-null  uint8
 6   APPLICATION_TYPE_T3           34299 non-null  uint8
 7   APPLICATION_TYPE_T4           34299 non-null  uint8
 8   APPLICATION_TYPE_T5           34299 non-null  uint8
 9   APPLICATION_TYPE_T6           34299 non-null  uint8
 10  APPLICATION_TYPE_T7           34299 non-null  uint8
 11  APPLICATION_TYPE_T8           34299 non-null  uint8
 12  AFFILIATION_CompanySponsored  34299 non-null  uint8
 13  AFFILIATION_Family/Parent     3

In [18]:
#checking for null values
has_null = application_df_processed.isnull().values.any()
print("Are there any null values in the DataFrame?", has_null)

Are there any null values in the DataFrame? False


In [19]:
# Split our preprocessed data into our features and target arrays
y = application_df_processed['IS_SUCCESSFUL'].values
X = application_df_processed.drop(columns='IS_SUCCESSFUL').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)


In [20]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer

nn.add(tf.keras.layers.Dense(units=16, activation='relu', input_dim=44))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=16, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 16)                720       
                                                                 
 dense_3 (Dense)             (None, 16)                272       
                                                                 
 dense_4 (Dense)             (None, 1)                 17        
                                                                 
Total params: 1009 (3.94 KB)
Trainable params: 1009 (3.94 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [28]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [29]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.5834 - accuracy: 0.7121
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5549 - accuracy: 0.7294
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5515 - accuracy: 0.7313
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5498 - accuracy: 0.7327
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5488 - accuracy: 0.7318
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5478 - accuracy: 0.7321
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5476 - accuracy: 0.7337
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5468 - accuracy: 0.7320
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5464 - accuracy: 0.7334
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5464 - accura

In [30]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5544 - accuracy: 0.7244 - 337ms/epoch - 1ms/step
Loss: 0.554397463798523, Accuracy: 0.7244315147399902


In [ ]:
# Export our model to HDF5 file
#  YOUR CODE GOES HERE

# **Making Adjustments to the Model**

In [45]:
# Split our preprocessed data into our features and target arrays
y1 = application_df_processed['IS_SUCCESSFUL'].values
X1 = application_df_processed.drop(columns=['IS_SUCCESSFUL','STATUS', 'SPECIAL_CONSIDERATIONS_N', 'SPECIAL_CONSIDERATIONS_Y']).values

# Split the preprocessed data into a training and testing dataset
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, random_state=42, stratify=y)


In [46]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X1_scaler = scaler.fit(X1_train)

# Scale the data
X1_train_scaled = X1_scaler.transform(X1_train)
X1_test_scaled = X1_scaler.transform(X1_test)

#Compile and Evaluate the Second Model

In [84]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn2 = tf.keras.models.Sequential()

# First hidden layer

nn2.add(tf.keras.layers.Dense(units=16, activation="tanh", input_dim=41))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=20, activation="tanh"))

# Third hidden layer
nn2.add(tf.keras.layers.Dense(units=20, activation="tanh"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_55 (Dense)            (None, 16)                672       
                                                                 
 dense_56 (Dense)            (None, 20)                340       
                                                                 
 dense_57 (Dense)            (None, 20)                420       
                                                                 
 dense_58 (Dense)            (None, 1)                 21        
                                                                 
Total params: 1453 (5.68 KB)
Trainable params: 1453 (5.68 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [85]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [86]:
# Train the model
fit_model2 = nn2.fit(X1_train_scaled, y1_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.5800 - accuracy: 0.7105
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5572 - accuracy: 0.7285
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5529 - accuracy: 0.7315
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5514 - accuracy: 0.7312
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5502 - accuracy: 0.7312
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5492 - accuracy: 0.7332
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5487 - accuracy: 0.7315
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5474 - accuracy: 0.7327
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5470 - accuracy: 0.7337
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5465 - accura

In [87]:
# Evaluate the model using the test data
model_loss1, model_accuracy1 = nn2.evaluate(X1_test_scaled,y1_test,verbose=2)
print(f"Loss: {model_loss1}, Accuracy: {model_accuracy1}")

268/268 - 0s - loss: 0.5560 - accuracy: 0.7241 - 301ms/epoch - 1ms/step
Loss: 0.556047797203064, Accuracy: 0.7240816354751587
